In [2]:
import requests
import pandas as pd
import numpy as np
import nbconvert
import datetime
import warnings; warnings.simplefilter('ignore')
import plotly.graph_objects as go
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import scrapbook as sb



ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Papermill variables to change to look at different buildings / organizations 
building_id = 2
server_web_address = 'https://bms.ahfc.us'

In [ ]:
# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 1)
sb.glue('title', 'Electric Use')

server = Server(server_web_address)
building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']
electric_id = building_df[0]['electric_ids']
current_building_type = building_df[0]['building_type']

if len(electric_id) == 0:
    error_message = 'This building does not appear to have any electric data. Either there is no data source or it has not been properly configured.'
    raise RuntimeError(error_message)

all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)

title_md = '''# Electricity Consumption for {} building'''
title_md = title_md.format(current_building_name)

In [ ]:
Markdown(title_md)

In [ ]:
fifteen_min_averages = server.sensor_readings((electric_id, 'electric_usage'),
                                          start_ts = datetime.datetime.now() - relativedelta(years=3),
                                            end_ts = datetime.datetime.now(),
                                          averaging = '15min')

# Get rid of any potential erroneous data that is listed as a negative electric usage
fifteen_min_averages = fifteen_min_averages.query("electric_usage >= 0")

fifteen_min_averages = fifteen_min_averages.reset_index()

fifteen_min_averages = fifteen_min_averages.rename(columns={'index':'datetime_col'})

fifteen_min_averages['dayofweek'] = fifteen_min_averages.datetime_col.apply(lambda x: x.dayofweek)

fifteen_min_averages['date'] = fifteen_min_averages.datetime_col.apply(lambda x: x.date())

fifteen_min_averages['time'] = fifteen_min_averages.datetime_col.apply(lambda x: x.time())

In [ ]:
current_week_start = fifteen_min_averages.date.iloc[-1] - pd.offsets.Day(7)

In [ ]:
# Create dataframes for each of the last three weeks for graphing comparison purposes.
last_week_start = current_week_start - pd.offsets.Day(7)
last_week_df = fifteen_min_averages.query("datetime_col >= @last_week_start")
last_week_df = last_week_df.query("datetime_col < @current_week_start")

two_weeks_ago_start = last_week_start - pd.offsets.Day(7)
two_weeks_ago_df = fifteen_min_averages.query("datetime_col >= @two_weeks_ago_start")
two_weeks_ago_df = two_weeks_ago_df.query("datetime_col < @last_week_start")

three_weeks_ago_start = two_weeks_ago_start - pd.offsets.Day(7)
three_weeks_ago_df = fifteen_min_averages.query("datetime_col >= @three_weeks_ago_start")
three_weeks_ago_df = three_weeks_ago_df.query("datetime_col < @two_weeks_ago_start")

In [ ]:
# Adjust the datetime column so the visualization will have the electricity data overlaying the current week.
last_week_df['adjusted_datetime_col'] = last_week_df.datetime_col.apply(lambda x: x + pd.offsets.Day(7))
two_weeks_ago_df['adjusted_datetime_col'] = two_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(14))
three_weeks_ago_df['adjusted_datetime_col'] = three_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(21))

In [ ]:
current_week = fifteen_min_averages.query("datetime_col >= @current_week_start")

In [ ]:
historical_complete = fifteen_min_averages.query("datetime_col < @current_week_start")

In [ ]:
day_avgs = historical_complete.groupby(['dayofweek', 'time']).mean()
day_avgs = day_avgs.reset_index()

In [ ]:
current_week_w_day_avgs = pd.merge(current_week, day_avgs, how='left',
                                  left_on=['dayofweek', 'time'],
                                  right_on=['dayofweek', 'time'], 
                                  suffixes=('', '_historical_avg'))

In [ ]:
current_week_w_day_avgs['current_use_vs_historical_difference'] = current_week_w_day_avgs.electric_usage - current_week_w_day_avgs.electric_usage_historical_avg

In [ ]:
# Define anomalous range as three standard deviations above the mean
three_std = current_week_w_day_avgs.current_use_vs_historical_difference.mean() + current_week_w_day_avgs.current_use_vs_historical_difference.std() * 3

In [ ]:
# This works, but the graph looks odd unless we get one data point on either side to connect them
current_week_w_day_avgs['electric_use_anomalies'] = np.where(current_week_w_day_avgs.current_use_vs_historical_difference > three_std,
                                                            current_week_w_day_avgs.electric_usage,
                                                            np.nan)

In [ ]:
anomaly_df = current_week_w_day_avgs.query("electric_use_anomalies == electric_use_anomalies")

In [ ]:
diff_series = np.diff(anomaly_df.index, n=1)

In [ ]:
diff_series = np.insert(diff_series, 0, 0)
anomaly_df['diff_series'] = diff_series

In [ ]:
# This keeps track of the number of groups of distinct anomalous periods in the data
group_counter = 0

# This padding is a cut-off used to group together time series indices that are close 
# enough to still be considered a group (e.g. there might be an anomaly at one time,
# followed by another anomalous reading 45 minutes later, which should probably all just
# be considered the same group)
index_padding = 4

for idx, row in anomaly_df.iterrows():
    if row['diff_series'] <= index_padding:
        anomaly_df.at[idx, 'group_number'] = group_counter
    else:
        group_counter += 1
        anomaly_df.at[idx, 'group_number'] = group_counter

In [ ]:
anomaly_max = anomaly_df.groupby(['group_number']).max()[['datetime_col', 'electric_usage']]
anomaly_min = anomaly_df.groupby(['group_number']).min()[['datetime_col', 'electric_usage']]

In [ ]:
anomaly_groups = pd.merge(anomaly_max, anomaly_min, how='outer',
                         left_index=True, right_index=True, 
                         suffixes=('_max', '_min'))

In [ ]:
if anomaly_groups.datetime_col_min.isna().all():
    anomaly_rectangles = []
    image_list = []
else:
    anomaly_rectangles = []
    image_list = []
    for idx, row in anomaly_groups.iterrows():
        
        anomaly_start = row['datetime_col_min']
        anomaly_end = row['datetime_col_max']
        anomaly_midpoint = ((anomaly_end - anomaly_start) / 2) + anomaly_start
        
        anomaly_rectangles.append({'type':'rect',
                            'xref':'x',
                            'yref':'paper',
                            'x0':anomaly_start,
                            'y0':0,
                            'x1':anomaly_end,
                            'y1':1,
                            'fillcolor':('rgb(205, 12, 24)'),
                            'opacity':0.5,
                            'line': {
                                'width':1,
                            }
                            })
        
        image_list.append({'source': 'https://github.com/dustin-cchrc/cchrc_python_for_non_programmers/blob/master/energy_savings_icon.png',
                'xref': "paper",
                'yref': "paper",
                'x': 1.0, #anomaly_midpoint ,
                'y': 0.9,
                'sizex': 0.2,
                'sizey': 0.2,
                'xanchor': "right",
                'yanchor': "bottom"})

In [ ]:
this_week = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                   y = current_week_w_day_avgs.electric_usage,
                   line = dict(color = '#ca0020'),
                   name='Current week electricity consumption')

last_week = go.Scatter(x = last_week_df.adjusted_datetime_col,
                      y = last_week_df.electric_usage,
                       line = dict(dash = 'dashdot',
                                   color = '#3182bd'),
                       name = "Electricity consumption last week")

two_weeks_ago = go.Scatter(x = two_weeks_ago_df.adjusted_datetime_col,
                          y = two_weeks_ago_df.electric_usage,
                           line = dict(dash = 'dashdot',
                                       color = '#6baed6'),
                           name = "Electricity consumption 2 weeks ago")

three_weeks_ago = go.Scatter(x = three_weeks_ago_df.adjusted_datetime_col,
                            y = three_weeks_ago_df.electric_usage,
                             line = dict(dash = 'dashdot',
                                         color = '#bdd7e7'),
                            name = "Electricity consumption 3 weeks ago")
    
layout = dict(title = 'Electricity Consumption: Current vs. Recent Weeks',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Electricity Consumption (kW, 15 minute average)')
             )

data = [this_week, last_week, two_weeks_ago, three_weeks_ago]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
this_week = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                   y = current_week_w_day_avgs.electric_usage,
                   line = dict(color = ('rgb(22, 96, 167)')),
                   name='Current Week Electricity Consumption')

historical_avg = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                   y = current_week_w_day_avgs.electric_usage_historical_avg,
                   line = dict(dash = 'dashdot',
                              color = ('rgb(22, 96, 167)')),
                            opacity = 0.6,
                   name='Historical Average Electricity Consumption')
    
layout = dict(title = 'Current Electricity Consumption vs. Historical Average',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Electricity Consumption (kW, 15 minute average)'),
              images = image_list,
              shapes = anomaly_rectangles
             )

data = [this_week, historical_avg]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
if anomaly_groups.datetime_col_min.isna().all():
    md_results = '''#### <font color='green'>There were no periods of extreme electricity consumption this week-- keep up the good work!</font>'''
else:
     md_results = '''#### <font color='red'>The periods highlighted in red signify much higher electricity consumption than normal; try to identify what happened and how to prevent it in the future.</font>'''

Markdown(md_results)

# Unoccupied electricity usage

In [ ]:
# Get schedule and timezone from the API data
building_schedule = building_df[0]['schedule']
building_timezone = building_df[0]['timezone']

# Create a schedule object using Ian's library
schedule_object = bmondata.Schedule(building_schedule, building_timezone)

if building_schedule == '' or schedule_object is None:
    md_results_2 = '''#### <font color='red'>There is no occcupied schedule entered for this building.</font>'''
else:
    # Set the start and end times of the graph
    graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
    graph_end_date = datetime.datetime.now()

    # Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
    list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

    # Loop through the list of occupied timestamps and convert them to datetimes
    start_time_list = []
    end_time_list = []

    for i in np.arange(0, len(list_of_occupied_timestamps)):
        start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
        start_time_list.append(start_datetime)
        end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
        end_time_list.append(end_datetime)

    # Create variables to indicate the first occupied start time and the last occupied end time
    occ_start = start_time_list[0]
    occ_end = end_time_list[-1]
    occ_day_list = np.arange(occ_start.weekday(), occ_end.weekday()+1)
    
    min_value = current_week_w_day_avgs.electric_usage.min()
    max_value = current_week_w_day_avgs.electric_usage.max()
    
    shape_list = []
    shape_dict = {'type':'rect',
                  'fillcolor':'#bdbdbd',
                  'opacity':0.35,
                  'line': {'width':1},
                  'layer':'below',
                  'y0':0,
                  'y1':max_value
                 }

    if graph_start_date < start_time_list[0]:
        shape_dict.update(x0=graph_start_date)
        shape_dict.update(x1=start_time_list[0])
        shape_list.append(shape_dict.copy())

    for i in np.arange(len(start_time_list)):
        if start_time_list[-1] != start_time_list[i]:
            shape_dict.update(x0=end_time_list[i])
            shape_dict.update(x1=start_time_list[i+1])
            shape_list.append(shape_dict.copy())

    if graph_end_date > end_time_list[-1]:
        shape_dict.update(x0=end_time_list[-1])
        shape_dict.update(x1=graph_end_date)
        shape_list.append(shape_dict.copy())
        
    electricity_usage = go.Scatter(x=current_week_w_day_avgs.datetime_col,
                             y=current_week_w_day_avgs.electric_usage,
                             name="Electricity Consumption")
    
    layout = dict(title='Current Week Electricity Consumption',
                  xaxis=dict(title='Date and Time'),
                  yaxis=dict(title='Electricity Consumption (kW, 15 min. avg))'),
                  shapes = shape_list
                 )


    fig = go.Figure(dict(data=[electricity_usage], layout=layout))
    
    md_results_2 = ''
    
    fig.show()

In [ ]:
Markdown(md_results_2)

###  Grey areas indicate that the building is unoccupied; if electricity consumption does not decrease significantly during these periods, check to see if:
 - Lighting is being left on at night
 - Ventilation fans are running at night
 - Other unnecessary equipment is being left on

## Ratio of night-time to daytime electricity usage

In [ ]:
today = datetime.date.today()
last_monday = today - datetime.timedelta(days=today.weekday())

In [ ]:
five_mondays_ago = last_monday - relativedelta(days=28)
five_mondays = fifteen_min_averages.query("datetime_col >= @five_mondays_ago")

In [ ]:
five_mondays['days_ago'] = five_mondays.date.apply(lambda x: (last_monday - x).days)

In [ ]:
five_mondays['weeks_ago'] = np.ceil(five_mondays.days_ago / 7)

In [ ]:
five_mondays['timestamp'] = five_mondays.datetime_col.apply(lambda x: datetime.datetime.timestamp(x))

In [ ]:
if schedule_object is None:
    five_mondays['occupied'] = 1
else: 
    five_mondays['occupied'] = five_mondays.timestamp.apply(lambda x: schedule_object.is_occupied(x, resolution='exact'))

In [ ]:
five_mondays['weekday'] = five_mondays.datetime_col.apply(lambda x: x.weekday())

In [ ]:
recent_weekdays = five_mondays.query("weekday < 5")

In [ ]:
recent_occupied_weekdays = recent_weekdays.query("occupied == True")

In [ ]:
recent_unoccupied_weeekdays = recent_weekdays.query("occupied == False")

In [ ]:
grouped_occupied = recent_occupied_weekdays.groupby(['weeks_ago']).mean()[['electric_usage']]

In [ ]:
grouped_unoccupied = recent_unoccupied_weeekdays.groupby(['weeks_ago']).mean()[['electric_usage']]

In [ ]:
mean_ratios = pd.merge(grouped_occupied, grouped_unoccupied, how='outer',
                       left_index=True, right_index=True, suffixes=['_occupied', '_unoccupied'])

In [ ]:
week_start_dates = recent_weekdays.groupby(['weeks_ago']).min()['datetime_col']

In [ ]:
mean_ratios_week_starts = pd.merge(mean_ratios, week_start_dates, how='left',
                                  left_index=True, right_index=True)
mean_ratios_week_starts['unoccupied_to_occupied_electricity_ratio'] = mean_ratios_week_starts.electric_usage_unoccupied / mean_ratios_week_starts.electric_usage_occupied 

In [ ]:
mean_ratios_week_starts['week_of_text'] = 'Week of ' + mean_ratios_week_starts.datetime_col.apply(lambda x: str(x.date()))

In [ ]:
max_x = mean_ratios_week_starts.datetime_col.max()+relativedelta(days=3)
min_x = mean_ratios_week_starts.datetime_col.min()-relativedelta(days=3)
midpoint = max_x - min_x
line_x = midpoint / 2 + min_x

bar1 = go.Bar(x=mean_ratios_week_starts.datetime_col,
              y=mean_ratios_week_starts.unoccupied_to_occupied_electricity_ratio, 
              orientation='v',
              text=mean_ratios_week_starts.week_of_text,
              textposition='auto')

textbox = go.Scatter(
        x=[line_x],
        y=[0.45],
        text=["<b>Typically Achievable Ratio for Alaskan Buildings</b>"],
        mode="text",
        textfont=dict(color="DarkSeaGreen",size=18)
)

data = [bar1, textbox]

layout = dict(title='Ratio of Unoccupied to Occupied Electricity Usage for ' + current_building_name + ': Weekday Averages',
              xaxis=dict(showticklabels=False), 
              yaxis=dict(title='Ratio of Unocccupied to Occupied Electricity Use (weekday averages)'),
             shapes= [dict(type="line",
                           x0=min_x, y0=0.5,
                           x1=max_x, y1=0.5,
                           line=dict(color="DarkSeaGreen",
                                     width=4))],
             showlegend=False)

fig = go.Figure(dict(data=data, layout=layout))

In [ ]:
if building_schedule == '' or schedule_object is None:
    md_results_3 = '''#### <font color='red'>There is no occcupied schedule entered for this building.</font>'''
    Markdown(md_results_3)
else: 
    fig.show()
    md_results_3 = ''

In [ ]:
Markdown(md_results_3)

<details>
  <summary>Click for details</summary>
  
  ## Ratio of Unoccupied to Occuppied Electricity Usage
  - **Description:** This graph shows the ratio of the unoccupied night-time electricity consumption to the occupied day time electricity consumption. It calculated the average weekday unoccupied electricity consumption (usually nighttime) and divided it by the average weekday occupied electricity consumption. 
  - **Potential for savings:** A ratio of one means that the building is using the same amount of electricity during unoccupied night-time periods as during the occupied day. This suggests opportunities for saving energy by turning off lights, appliances, ventilation systems, etc. at night. Higher numbers signify a higher potential savings from turning equipment off at night. 

</details>

# Electricity usage compared to other offices 

In [ ]:
all_buildings_of_type = org_df.query("building_type == @current_building_type")

In [ ]:
all_buildings_of_type_list = []

for elec_id in all_buildings_of_type.electric_ids.unique():
    if elec_id != '':
        temp_df = server.sensor_readings((elec_id, 'electric_usage'),
                                              start_ts = datetime.datetime.now() - relativedelta(years=3),
                                                end_ts = datetime.datetime.now(),
                                              averaging = '1H')
        temp_df['building_name'] = all_buildings_of_type.query("electric_ids == @elec_id").iloc[0]['title']
        temp_df['building_square_footage'] = all_buildings_of_type.query("electric_ids == @elec_id").iloc[0]['floor_area']
        all_buildings_of_type_list.append(temp_df)

all_buildings_of_type_data = pd.concat(all_buildings_of_type_list)

In [ ]:
# Sum the kWh (average kW over the course of an hour) by month for each building
monthly_grouped_data = all_buildings_of_type_data.groupby(['building_name', lambda x:x.year, lambda x: x.month]).sum()
monthly_grouped_data = monthly_grouped_data.reset_index()
monthly_grouped_data = monthly_grouped_data.rename(columns={'level_1':'year',
                                                           'level_2':'month',
                                                           'electric_usage':'total_monthly_kwh'})
monthly_grouped_data = monthly_grouped_data.drop(columns='building_square_footage', axis=1)

In [ ]:
# Join the original building square footage data back to the dataset
all_buildings_of_type_data = all_buildings_of_type_data[['building_name', 'building_square_footage']]
all_buildings_of_type_data = all_buildings_of_type_data.drop_duplicates()
all_buildings_of_type_data = pd.merge(monthly_grouped_data, all_buildings_of_type_data, how='left',
                                     left_on='building_name', right_on='building_name')

In [ ]:
# Normalize by square footage. Data starts as kwh per month; result is in kwh / square foot / month
all_buildings_of_type_data['monthly_electricity_eui'] = all_buildings_of_type_data.total_monthly_kwh / all_buildings_of_type_data.building_square_footage

In [ ]:
all_buildings_of_type_data['datetime'] = pd.to_datetime(dict(year=all_buildings_of_type_data.year, month=all_buildings_of_type_data.month, day=1))

In [ ]:
diverging_hues = ['#d73027','#fc8d59',
                  '#fee090','#ffffbf',
                  '#e0f3f8','#91bfdb',
                  '#4575b4']

In [ ]:
building_type_dict = {'OFFIC':'Office', 
                      'SCH':'School', 
                      'M-RES':'Multifamily Residential', 
                      'OTHER':'Miscellaneous Type'}

In [ ]:
current_building_type = building_type_dict[current_building_type]

In [ ]:
current_building_df = all_buildings_of_type_data.query("building_name == @current_building_name")
all_buildings_avg_df = all_buildings_of_type_data.query("building_name != @current_building_name")
all_buildings_avg_df = all_buildings_avg_df.groupby(['year', 'month']).mean()
all_buildings_avg_df = all_buildings_avg_df.reset_index()
all_buildings_avg_df['datetime'] = pd.to_datetime(dict(year=all_buildings_avg_df.year, month=all_buildings_avg_df.month, day=1))

In [ ]:
all_building_avg_monthly_electric_eui = go.Scatter(x=all_buildings_avg_df.datetime,
                                                   y=all_buildings_avg_df.monthly_electricity_eui,
                         name="Average of Alaska " + current_building_type + " buildings")

current_building = go.Scatter(x=current_building_df.datetime,
                             y=current_building_df.monthly_electricity_eui,
                             name=current_building_name)

layout = dict(title= 'Monthly Electricity EUI comparison: ' + current_building_name + ' compared to other ' + current_building_type + ' buildings',
              xaxis=dict(title='Date'),
              yaxis=dict(title='Electricity energy use intensity (kWh / square foot / month)')
             )

fig = go.Figure(dict(data=[all_building_avg_monthly_electric_eui, current_building], layout=layout))

In [ ]:
if current_building_df.building_square_footage.isna().all():
    markdown_error = '''#### <font color='red'>This building has no square footage data, and so cannot be compared to other buildings.</font>'''
else: 
    fig.show()
    markdown_error = ''

In [ ]:
Markdown(markdown_error)